In [163]:
import numpy as np
import pandas as pd
import matplotlib as plt
from IPython.display import clear_output


In [164]:
#import tensorflow.compat.v2.feature_column as fc

In [165]:
import tensorflow as tf

In [166]:
dftrain=pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
dfeval=pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/eval.csv")
y_train=dftrain.pop('survived')  #remove label column
y_eval=dfeval.pop('survived')     #remove label column

In [181]:
dftrain


,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
...,...,...,...,...,...,...,...,...,...
622,male,28.0,0,0,10.5000,Second,unknown,Southampton,y
623,male,25.0,0,0,7.0500,Third,unknown,Southampton,y
624,female,19.0,0,0,30.0000,First,B,Southampton,y
625,female,28.0,1,2,23.4500,Third,unknown,Southampton,n


In [168]:
y_train[0].shape

()

In [169]:
categorical_columns=['sex','n_siblings_spouses','parch','class','deck','embark_town','alone']

In [170]:
numeric_columns=['age','fare']

In [171]:
feature_columns=[] #columns we need to feed to our modelto get predictions

In [172]:
for feature_name in categorical_columns:
    vocabulary=dftrain[feature_name].unique() 
    #gets a list of all unique values from given feature column
    #print(vocabulary)
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name,vocabulary))
    
    #this randomly gives the unique string in each feature column values from0,1,2,3.. based on the index position in vocabulary

In [173]:
for feature_name in numeric_columns:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype=tf.float32))

In [174]:
print(feature_columns)

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, def

In [175]:
#this makes an input function and returns the object of input function for use
def make_input_fn(data_df,label_df,num_epochs=10,shuffle=True,batch_size=32):
    def input_function():   #inner function ,this will be returned
        #model needs data in tf.data.Dataset object to create a model
        #this creates a tf.data.Dataset object with data and its label
        ds=tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))
        if shuffle:
            #In each repetition, this dataset fills a buffer with buffer_size elements, then randomly samples elements from this buffer, replacing the selected elements with new elements. For perfect shuffling, set the buffer size equal to the full size of the dataset.
            #For instance, if your dataset contains 10,000 elements but buffer_size is set to 1,000, then shuffle will initially select a random element from only the first 1,000 elements in the buffer. Once an element is selected, its space in the buffer is replaced by the next (i.e. 1,001-st) element, maintaining the 1,000 element buffer.
            ds=ds.shuffle(1000) #randomize order of data and 1000 is the buffer_size
        
        #split dataset into batches of 32 and repeat process for numer of epochs
        ds=ds.batch(batch_size).repeat(num_epochs)
        return ds  #return batch of the datset 
    return input_function   #return a function object for use

train_input_fn=make_input_fn(dftrain,y_train)
eval_input_fn=make_input_fn(dfeval,y_eval,num_epochs=1,shuffle=False)
                    

In [176]:
linear_est=tf.estimator.LinearClassifier(feature_columns=feature_columns)
#all models use estimator which are inbuilt algorithms and we pass the feature_columns that is what to expect in each column 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\hp\\AppData\\Local\\Temp\\tmpeks43ley', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [177]:
linear_est.train(train_input_fn)   #train
result=linear_est.evaluate(eval_input_fn)   #get model metrics/stats 

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\hp\AppData\Local\Temp\tmpeks43ley\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss 

In [178]:
print(result["accuracy"])

0.7386364


In [179]:
print(result)

{'accuracy': 0.7386364, 'accuracy_baseline': 0.625, 'auc': 0.8364248, 'auc_precision_recall': 0.789361, 'average_loss': 0.48135632, 'label/mean': 0.375, 'loss': 0.474348, 'precision': 0.64705884, 'prediction/mean': 0.40438586, 'recall': 0.6666667, 'global_step': 200}


In [180]:
result= list(linear_est.predict(eval_input_fn))
print(dfeval.loc[0])
print(y_eval.loc[0])
print(result[0]['probabilities'][1])    #0 is not survived 1 is they survived

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\hp\AppData\Local\Temp\tmpeks43ley\model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
sex                          male
age                            35
n_siblings_spouses              0
parch                           0
fare                         8.05
class                       Third
deck       

In [317]:
q=dftrain.loc[0]
q={'sex': 'male', 'age': 55.0, 'n_siblings_spouses': 2, 'parch': 2, 'fare': 7.25, 'class': 'First', 'deck': 'C', 'embark_town': 'Southampton', 'alone': 'y'}
q=pd.Series(q)
d=dfeval.append(q,ignore_index=True)
d1=y_eval.append(pd.Series(-1))
q

sex                          male
age                            55
n_siblings_spouses              2
parch                           2
fare                         7.25
class                       First
deck                            C
embark_town           Southampton
alone                           y
dtype: object

In [318]:
eval_input_fn1=make_input_fn(d,d1,num_epochs=1,shuffle=False)
result= list(linear_est.predict(eval_input_fn1))
result[-1]['probabilities']


INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\hp\AppData\Local\Temp\tmpeks43ley\model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


array([0.9094101 , 0.09058987], dtype=float32)